- Normalização: `letreiro` para `UPPER(TRIM)`, `cod_linha` cast `int`.
- *Left join* com `bronze_gtfs_routes` por `route_short_name` (normalizado) quando aplicável.
- **Observação**: como o letreiro público (`7021-10`) nem sempre bate 1:1 com o `route_short_name` do GTFS, campos GTFS podem ficar `NULL` (esperado em alguns casos).

In [0]:
from pyspark.sql.functions import trim, upper, col, regexp_extract, concat_ws

bronze_linhas = spark.table("workspace.sptrans.bronze_linhas")
gtfs_routes = spark.table("workspace.sptrans.bronze_gtfs_routes")

# ✅ 1) Reconstrói letreiro completo
api_clean = (
    bronze_linhas
    .withColumn("letreiro", trim(upper(col("letreiro"))))
    .withColumn("letreiro_decimal", trim(upper(col("letreiro_decimal"))))
    .withColumn("letreiro_full", concat_ws("-", col("letreiro"), col("letreiro_decimal")))
    .withColumn("letreiro_norm", regexp_extract(col("letreiro"), r'([0-9]+)', 1))  # só a parte numérica
    .withColumn("cod_linha", col("cod_linha").cast("int"))
    .dropna(subset=["cod_linha"])
)

gtfs_clean = (
    gtfs_routes
    .withColumn("route_short_name_norm", regexp_extract(upper(trim(col("route_short_name"))), r'([0-9]+)', 1))
)

silver_linhas = (
    api_clean.alias("api")
    .join(
        gtfs_clean.alias("gtfs"),
        col("api.letreiro_norm") == col("gtfs.route_short_name_norm"),
        "left"
    )
    .select(
        col("api.cod_linha"),
        col("api.letreiro_full").alias("letreiro"),
        col("api.letreiro_norm").alias("letreiro_num"),
        col("api.sentido"),
        col("gtfs.route_id"),
        col("gtfs.route_long_name").alias("nome_oficial"),
        col("gtfs.agency_id")
    )
    .dropDuplicates(["cod_linha"])
)

silver_linhas.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.sptrans.silver_linhas")

print("✅ silver_linhas reconstruída e enriquecida!")
display(silver_linhas.limit(20))


✅ silver_linhas reconstruída e enriquecida!


cod_linha,letreiro,letreiro_num,sentido,route_id,nome_oficial,agency_id
34514,5031-10,5031,2,5031-21,Vl. Cde. Do Pinhal - Term. Sacomã,1
33342,8215-10,8215,2,8215-10,Jd. Paulistano - Pça. Do Correio,1
34515,5031-21,5031,2,5031-21,Vl. Cde. Do Pinhal - Term. Sacomã,1
803,819A-10,819,1,819A-10,Cptm Vl. Aurora - Jd. Primavera,1
2543,3010-10,3010,1,3010-10,Cem. Da Saudade - Term. A. E. Carvalho,1
738,3390-10,3390,1,3390-10,São Mateus - Term. Pq. D. Pedro Ii,1
32954,6013-10,6013,2,6013-21,Etec - Jd. âNgela,1
35041,119L-10,119,2,119L-10,Vl. Sulina - Term. Lapa,1
34240,6003-10,6003,2,6003-21,Term. Varginha - Est. Mendes/vl. Natal,1
33233,958P-10,958,2,958P-10,Jd. Nardini - Vl. Olímpia,1


In [0]:
%sql
SELECT *
FROM workspace.sptrans.silver_linhas
WHERE route_id IS NULL;

In [0]:
from pyspark.sql.functions import trim, upper, col

bronze_linhas = spark.table("workspace.sptrans.bronze_linhas")
gtfs_routes = spark.table("workspace.sptrans.bronze_gtfs_routes")

# limpeza e padronização
api_clean = (
    bronze_linhas
    .withColumn("letreiro", trim(upper(col("letreiro"))))
    .withColumn("cod_linha", col("cod_linha").cast("int"))
    .dropna(subset=["cod_linha"])
)

gtfs_clean = (
    gtfs_routes
    .withColumn("route_short_name", trim(upper(col("route_short_name"))))
)

silver_linhas = (
    api_clean.alias("api")
    .join(
        gtfs_clean.alias("gtfs"),
        col("api.letreiro") == col("gtfs.route_short_name"),
        "left"
    )
    .select(
        col("api.cod_linha"),
        col("api.letreiro"),
        col("api.sentido"),
        col("gtfs.route_id"),
        col("gtfs.route_long_name").alias("nome_oficial"),
        col("gtfs.agency_id"),
    )
    .dropDuplicates(["cod_linha"])
)

silver_linhas.write \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("workspace.sptrans.silver_linhas")

print("✅ silver_linhas criada com sucesso!")
display(silver_linhas.limit(10))


✅ silver_linhas criada com sucesso!


cod_linha,letreiro,sentido,route_id,nome_oficial,agency_id
34514,5031,2,null,null,null
33342,8215,2,null,null,null
34515,5031,2,null,null,null
803,819A,1,null,null,null
2543,3010,1,null,null,null
738,3390,1,null,null,null
32954,6013,2,null,null,null
35041,119L,2,null,null,null
34240,6003,2,null,null,null
33233,958P,2,null,null,null


In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM workspace.sptrans.silver_linhas;